In [2]:
from model_encoder import *
from optim_draw import *

ModuleNotFoundError: No module named 'torchvision'

# Drawing simple geometric figures

The target image:

In [1]:
imshow(image_loader("./Images/losange.jpg"))

NameError: name 'imshow' is not defined

The drawing:

In [ ]:
drawer = run(image_loader("./Images/losange.jpg"), 10, 50, clean=True, save=False, save_title='losange')

The target image:

In [ ]:
imshow(image_loader("./Images/ovale.jpg"))

The drawing:

In [ ]:
drawer = run(image_loader("./Images/ovale.jpg"), 50, 50, clean=True, save=False, save_title='ovale')

## Combining thin lines to create thicker lines

An interesting discovery was that when the lines in the targer image were wider than the ones used by the drawer, it would add many lines to create something that looks like the original. Here is an example where it had to do that:

In [ ]:
imshow(image_loader("./Images/texte.jpg"))

In [ ]:
drawer = run(image_loader("./Images/texte.jpg"), 150, 50, clean=True, save=False, save_title='hey')

## Coloring monochrome surfaces

The Drawer also intuitively colors monochrome surfaces by adding multiple lines, just as we would do with a pencil. Here is an example of this behaviour on a targer image:

In [ ]:
imshow(image_loader("./Images/ovale plein.jpg"))

In [ ]:
drawer = run(image_loader("./Images/ovale plein.jpg"), 250, 50, clean=True, save=False, save_title='ovale plein')

# Drawing simplified images

Before we try to draw complex images like real colored pictures, we first test the drawer on a simplified version of a picture of a boat. We feed it a black and white picture whose composed of straight lines only. Here is what it looks like:

In [ ]:
imshow(image_loader("./Images/stickboat.jpg"))

In [ ]:
drawer = run(image_loader("./Images/stickboat.jpg"), 250, 50, clean=True, save=False, save_title='stickboat')

# Drawing complex images

In [ ]:
imshow(image_loader("./Images/boat.jpg"))

In [ ]:
drawer = run(image_loader("./Images/boat.jpg"), 250, 50, clean=True, save=False, save_title='boat')